<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

В данном проекте создадим модель, определяющую, токсичен ли комментарий или нет. Оценим ее по критерию качества f1 (не меньше 0.75).
Построим несколько моделей и выберем лучшую.

## Подготовка

In [ ]:
import pandas as pd
import re
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
import spacy
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Рассмотрим данные.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


У нас есть столбец с текстом коментария text, столбец с классификацией toxic(токсичен или нет), и столбец, дублирующий индекс Unnamed: 0. Удалим этот столбец.

In [ ]:
data = data.drop('Unnamed: 0', axis=1)

Выполним леммитизацию и очистку текста от лишних символов.

In [ ]:
def clear_text(text):
    pattern = r'[^a-zA-Z]'
    cl_text = re.sub(pattern, ' ', text)
    return ' '.join(cl_text.split())

In [ ]:
data['lemms'] = data['text'].apply(clear_text)

In [ ]:
data.head()

,text,toxic,lemms
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [ ]:
def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
data.head()

,text,toxic,lemms
0,Explanation\nWhy the edits made under my usern...,0,Explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D aww he match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [ ]:
data = data.drop('text', axis=1)

Разделим данные на обучающую и тестовую выборку. Столбец с текстом у нас уже отделен на этапе лемитизации. Целевой столбец - toxic.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data['lemms'], data['toxic'], test_size=0.1, random_state=189)

Создадим "мешок слов". Переведем данные в векторный вид.

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
xtf_train = count_tf_idf.fit_transform(x_train)
xtf_test = count_tf_idf.transform(x_test)

In [ ]:
xtf_test.shape

(15930, 147646)

In [ ]:
xtf_train.shape

(143362, 147646)

## Обучение

Обучим модель - логистическую регрессию.

In [ ]:
model_lr = LogisticRegression(max_iter=1000, random_state=189, class_weight='balanced')

In [ ]:
scores_list = cross_val_score(estimator=model_lr, 
                              X=xtf_train, 
                              y=y_train, 
                              cv=5,  
                              scoring='f1')

val_score = scores_list.mean()
val_score

0.7535436686120482

In [ ]:
param_search = {}
gs_lr = GridSearchCV(estimator=model_lr, cv=5, param_grid=param_search, scoring='f1')
gs_lr.fit(xtf_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=1000, random_state=189),
             param_grid={}, scoring='f1')

In [ ]:
f1_lr = gs_lr.best_score_
f1_lr

0.7535436686120482

Применим модель - дерево решений.

In [ ]:
clf = DecisionTreeClassifier(random_state=189, class_weight='balanced')
parametrs = {'max_depth': range(1, 8)
              }
grid = GridSearchCV(clf, parametrs, cv=5, scoring='f1')
grid.fit(xtf_train, y_train)
grid.best_params_

{'max_depth': 9}

In [ ]:
f1_dtc = grid.best_score_
f1_dtc

0.5673970828581796

Рассчитаем f-меру.

Протестируем модель, показавшую лучший результат - логистическую регрессию.

In [ ]:
test_pred = gs_lr.predict(xtf_test)
f1_test = f1_score(y_test, test_pred)
f1_test

0.7538298996302166

На тестовой выборке модель тоже показала хороший результат.

## Выводы

In [ ]:
col = {
    'Model': ['DecisionTree', 'LogisticRegression'],
    'f1': [f1_dtc, val_score],
    }
final_scoring = pd.DataFrame(data=col)
final_scoring

,Model,f1
0,DecisionTree,0.560462
1,LogisticRegression,0.753544


В данном проекте мы подобрали модель, определяющую, токсичен ли комментарий или нет. У нас был датасэт с комментариями и меткой.

На этапе предобработки мы леммитизировали текст и привели данные к векторному виду.

На этапе обучения мы рассмотрели несколько моделей:
- DecisionTree
- LogisticRegression

Лучший результат показала логистическая регрессия = 0.75.